In [2]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.3MB/s 


In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
import zipfile
from tqdm import trange,tqdm
dset = zipfile.ZipFile('drive/Data/UTKFace.zip')
ilist = dset.infolist()
for i in trange(len(ilist)):
    dset.extract(ilist[i])

100%|██████████| 23708/23708 [00:06<00:00, 3773.91it/s]


# Race Prediction on LFW

## Import Libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential,load_model
from keras.layers import Convolution2D,MaxPooling2D,Dense,Dropout,Activation,Flatten
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard
from keras.utils import to_categorical
import seaborn as sns
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
from sklearn import svm
import random
import h5py
import cv2
from glob import glob
from tqdm import tqdm,trange
import re

Using TensorFlow backend.


In [0]:
num_epochs=50
batch_size = 64

## Load Data

In [0]:
filelist = glob("UTKFace/*.jpg")
labels = [re.search(r'UTKFace/(\d+)_(\d+)_(\d+).*',i).groups() for i in filelist]
labels = [[int(j) for j in i] for i in labels]
labels = np.array(labels)

In [0]:
def load_image(filepath):
    img = cv2.imread(filepath)
    img = img[25:-25,25:-25,:]
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(64,64))
    return img.astype(np.float32)/255.0

In [0]:
X = np.array([load_image(i) for i in filelist])

In [0]:
y = labels[:,2]
np.unique(y,return_counts=True)
lb = LabelBinarizer()
y = lb.fit_transform(y)

## VGG-16 Model

In [52]:
model = Sequential()
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3),padding='same'))
model.add(Convolution2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 128)       73856     
__________

In [0]:
es_callback = EarlyStopping(patience=3,verbose=1)
callback_list = [es_callback]

## Training the Model

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [54]:
model.fit(X_train,y_train,epochs=10,verbose=1,validation_data=(X_test,y_test))

Train on 18966 samples, validate on 4742 samples
Epoch 1/10
18966/18966 [==============================] - 59s 3ms/step - loss: 1.1870 - acc: 0.5550 - val_loss: 0.9208 - val_acc: 0.6664
Epoch 2/10
 1632/18966 [=>............................] - ETA: 50s - loss: 0.9313 - acc: 0.6716

18966/18966 [==============================] - 58s 3ms/step - loss: 0.8629 - acc: 0.6962 - val_loss: 0.8144 - val_acc: 0.7098
Epoch 3/10
 7872/18966 [===========>..................] - ETA: 32s - loss: 0.7783 - acc: 0.7329

18966/18966 [==============================] - 58s 3ms/step - loss: 0.7601 - acc: 0.7355 - val_loss: 0.7691 - val_acc: 0.7292
Epoch 4/10
10240/18966 [===============>..............] - ETA: 25s - loss: 0.6871 - acc: 0.7608

18966/18966 [==============================] - 58s 3ms/step - loss: 0.6864 - acc: 0.7590 - val_loss: 0.6793 - val_acc: 0.7644
Epoch 5/10
11136/18966 [================>.............] - ETA: 22s - loss: 0.6199 - acc: 0.7830

18966/18966 [==============================] - 58s 3ms/step - loss: 0.6218 - acc: 0.7825 - val_loss: 0.6636 - val_acc: 0.7706
Epoch 6/10
11488/18966 [=================>............] - ETA: 21s - loss: 0.5538 - acc: 0.8060

18966/18966 [==============================] - 57s 3ms/step - loss: 0.5636 - acc: 0.7999 - val_loss: 0.7039 - val_acc: 0.7693
Epoch 7/10
11616/18966 [=================>............] - ETA: 21s - loss: 0.4971 - acc: 0.8232

18966/18966 [==============================] - 58s 3ms/step - loss: 0.5030 - acc: 0.8225 - val_loss: 0.6733 - val_acc: 0.7754
Epoch 8/10
11648/18966 [=================>............] - ETA: 21s - loss: 0.4380 - acc: 0.8431

18966/18966 [==============================] - 57s 3ms/step - loss: 0.4439 - acc: 0.8427 - val_loss: 0.7018 - val_acc: 0.7733
Epoch 9/10
11680/18966 [=================>............] - ETA: 21s - loss: 0.3828 - acc: 0.8667

18966/18966 [==============================] - 57s 3ms/step - loss: 0.3922 - acc: 0.8643 - val_loss: 0.7275 - val_acc: 0.7758
Epoch 10/10
11648/18966 [=================>............] - ETA: 21s - loss: 0.3315 - acc: 0.8820

18966/18966 [==============================] - 57s 3ms/step - loss: 0.3416 - acc: 0.8807 - val_loss: 0.7233 - val_acc: 0.7725


In [0]:
model.load_weights("../Models/LFW_GRAYSCALE_64/Race/no-dropout-001-0.7030.hdf5")
y_pred = model.predict(X_train)

In [0]:
y_pred_int = []
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if y_pred[i][j]==max(y_pred[i]):
            y_pred_int.append(j)
            break

## Results

In [0]:
print(accuracy_score(y_train,y_pred_int))

0.81039336529


In [0]:
import os
os.path.exists('lfw-deepfunneled\\Aaron_Eckhart')

False

In [0]:
!ls lfw-deepfunneled/Aaron_Eckhart

Aaron_Eckhart_0001.jpg


In [0]:
type(model)

keras.models.Sequential

In [0]:
y_pred = model.predict(X_test)

In [0]:
def onehot_to_cat(y_pred):
  y_pred_int = []
  for i in y_pred:
    mi = max(i)
    for j in range(8):
      if(i[j]==mi):
        y_pred_int.append(j)
        break
  return np.array(y_pred_int)

In [0]:
y_pred_int = onehot_to_cat(y_pred)
y_true_int = onehot_to_cat(y_test)

In [58]:
print(classification_report(y_true_int,y_pred_int))

             precision    recall  f1-score   support

          0       0.82      0.84      0.83      2003
          1       0.83      0.81      0.82       896
          2       0.80      0.85      0.82       689
          3       0.69      0.73      0.71       818
          4       0.35      0.25      0.29       334
          6       0.00      0.00      0.00         1
          7       0.00      0.00      0.00         1

avg / total       0.76      0.77      0.77      4742



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
del lfwdf

In [13]:
print(re.search(r'UTKFace/(\d+)_(\d+)_(\d+).*',filelist[0]).groups())

('26', '1', '3')


In [14]:
filelist[0]

'UTKFace/26_1_3_20170117153635595.jpg.chip.jpg'

In [23]:
np.unique(labels[:,2],return_counts=True)

(array([                0,                 1,                 2,
                        3,                 4, 20170109142408075,
        20170109150557335, 20170116174525125]),
 array([10078,  4526,  3434,  3975,  1692,     1,     1,     1]))

In [24]:
X

array([[26,  1,  3],
       [68,  0,  0],
       [26,  1,  0],
       ...,
       [28,  0,  4],
       [90,  1,  1],
       [32,  1,  2]])

In [0]:
model.save("drive/Models/LFW_RGB_64/Race/RaceNet.hdf5")